In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/forgery-dataset/Forgery Dataset/valid/labels.cache
/kaggle/input/forgery-dataset/Forgery Dataset/valid/labels/image-5-_png_jpg.rf.e5780a684dd168381770ca4119dec735.txt
/kaggle/input/forgery-dataset/Forgery Dataset/valid/labels/image-5-_png_jpg.rf.ac3bc2b9ac58bae8422d870d746d0cfb.txt
/kaggle/input/forgery-dataset/Forgery Dataset/valid/labels/image-7-_png.rf.d7b2b4a701a414dd7d0295cea317d08c.txt
/kaggle/input/forgery-dataset/Forgery Dataset/valid/labels/image-5-_png.rf.d98fd2aca147040db84ac8145e74f10a.txt
/kaggle/input/forgery-dataset/Forgery Dataset/valid/labels/image-5-_png.rf.636a9b27d330102cf4477ce59ab5af1d.txt
/kaggle/input/forgery-dataset/Forgery Dataset/valid/labels/image-6-_png_jpg.rf.ef31b487d745dabe96c1b8704f0c78a3.txt
/kaggle/input/forgery-dataset/Forgery Dataset/valid/labels/image-7-_png.rf.9cee7faf44892d02c4c2f910852239b5.txt
/kaggle/input/forgery-dataset/Forgery Dataset/valid/labels/image-6-_png.rf.af3e4829b5cd5c2534de3e12709fd505.txt
/kaggle/input/forgery-datas

In [2]:
!pip install ultralytics

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.0/62.0 kB 2.2 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of mkl-fft to determine which version is compatible with other requirements. This could take a while.
INFO: pip is looking at multiple versions of mkl-random to determine which version is compatible with other requirements. This could take a while.
INFO: pip is looking at multiple versions of mkl-umath to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 21.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.8/16.8 MB 74.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 71.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 61.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 29.0 MB/s eta 0:00:00
   ━━━━━

In [ ]:
import os
import yaml
import torch
import random
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
from ultralytics import RTDETR
from sklearn.metrics import precision_recall_fscore_support
import cv2
import uuid

# Set random seed for reproducibility
torch.manual_seed(42)
np.random.seed(42)
random.seed(42)

# Dataset paths
dataset_path = "/kaggle/input/forgery-dataset/Forgery Dataset"
train_img_dir = f"{dataset_path}/train/images"
train_label_dir = f"{dataset_path}/train/labels"
val_img_dir = f"{dataset_path}/valid/images"
val_label_dir = f"{dataset_path}/valid/labels"
test_img_dir = f"{dataset_path}/test/images"
test_label_dir = f"{dataset_path}/test/labels"

# Verify dataset paths
for path in [train_img_dir, train_label_dir, val_img_dir, val_label_dir, test_img_dir, test_label_dir]:
    assert os.path.exists(path), f"Path {path} does not exist!"
print("✅ Dataset paths verified!")

# Create YAML configuration
yaml_config = {
    "path": dataset_path,
    "train": "train/images",
    "val": "valid/images",
    "test": "test/images",
    "names": {0: "forgery"},
    "nc": 1,
    "augment": True,
    "mosaic": 0.5,  # Moderate augmentation
    "scale": 0.5,
    "flipud": 0.5,
    "fliplr": 0.5
}

# Save YAML file
os.makedirs("/kaggle/working", exist_ok=True)
yaml_path = "/kaggle/working/c2a_rtdetr.yaml"
with open(yaml_path, "w") as f:
    yaml.dump(yaml_config, f)

# Verify YAML
with open(yaml_path, "r") as f:
    print("YAML Content:\n", f.read())

# Load RT-DETR model
try:
    model = RTDETR("rtdetr-l.pt")  # Pre-trained large model
    print("✅ RT-DETR model loaded successfully!")
except Exception as e:
    raise Exception(f"Failed to load RT-DETR model: {str(e)}")

# Training
device = "cuda:0" if torch.cuda.is_available() else "cpu"
results = model.train(
    data=yaml_path,
    epochs=20,  # Reduced for Kaggle resource limits
    imgsz=640,
    batch=8,
    device=device,
    project="/kaggle/working",
    name="rtdetr_forgery_detection",
    exist_ok=True,
    patience=5,  # Early stopping
    optimize=True,
    pretrained=True,
    cos_lr=True,
    lr0=0.001,
    augment=True,
    save=True,
    plots=True
)

# Verify checkpoint
checkpoint_path = "/kaggle/working/forgery_Detection/weights/best.pt"
assert os.path.exists(checkpoint_path), f"Training failed - no checkpoint at {checkpoint_path}"
print(f"✅ Model checkpoint saved at {checkpoint_path}")

# Validation
try:
    val_results = model.val(
        data=yaml_path,
        split="test",
        imgsz=640,
        batch=8,
        device=device,
        plots=True,
        save_json=True,
        project="/kaggle/working",
        name="rtdetr_validation"
    )
    print("✅ Validation completed!")
    
    # Extract metrics
    metrics = val_results.results_dict
    print(f"Validation Metrics:")
    print(f"Precision: {metrics['metrics/precision(B)']:.4f}")
    print(f"Recall: {metrics['metrics/recall(B)']:.4f}")
    print(f"mAP@50: {metrics['metrics/mAP50(B)']:.4f}")
except Exception as e:
    print(f"Validation failed: {str(e)}")
    # Fallback manual validation
    print("Running manual validation...")
    test_images = [f for f in os.listdir(test_img_dir) if f.endswith('.png')]
    metrics = {"tp": 0, "fp": 0, "fn": 0}
    
    for img_file in tqdm(test_images[:50], desc="Manual Validation"):
        img_path = os.path.join(test_img_dir, img_file)
        label_path = os.path.join(test_label_dir, img_file.replace('.png', '.txt'))
        
        # Run prediction
        pred = model.predict(img_path, conf=0.5)[0]
        
        # Compare with ground truth
        with open(label_path) as f:
            gt_boxes = len(f.readlines())
        pred_boxes = len(pred.boxes)
        metrics["tp"] += min(pred_boxes, gt_boxes)
        metrics["fp"] += max(0, pred_boxes - gt_boxes)
        metrics["fn"] += max(0, gt_boxes - pred_boxes)
    
    precision = metrics["tp"] / (metrics["tp"] + metrics["fp"] + 1e-10)
    recall = metrics["tp"] / (metrics["tp"] + metrics["fn"] + 1e-10)
    print(f"\nManual Validation Metrics (50 images):")
    print(f"Precision: {precision:.4f}, Recall: {recall:.4f}")
    print(f"TP: {metrics['tp']}, FP: {metrics['fp']}, FN: {metrics['fn']}")

# Prediction visualization
def plot_predictions(model, image_dir, n=3):
    image_files = [f for f in os.listdir(image_dir) if f.endswith('.png')]
    selected_images = random.sample(image_files, min(n, len(image_files)))
    
    for img_file in selected_images:
        img_path = os.path.join(image_dir, img_file)
        results = model.predict(img_path, conf=0.5)
        
        for r in results:
            plt.figure(figsize=(10, 10))
            img = r.plot()[:, :, ::-1]  # BGR to RGB
            plt.imshow(img)
            plt.axis('off')
            plt.title(f"Prediction: {img_file}")
            plt.savefig(f"/kaggle/working/prediction_{img_file}")
            plt.show()

# Run predictions on test set
plot_predictions(model, test_img_dir)

# Save error cases
error_dir = "/kaggle/working/error_analysis"
os.makedirs(error_dir, exist_ok=True)
test_images = [f for f in os.listdir(test_img_dir) if f.endswith('.png')][:20]  # Sample 20
for img_file in test_images:
    img_path = os.path.join(test_img_dir, img_file)
    label_path = os.path.join(test_label_dir, img_file.replace('.png', '.txt'))
    
    pred = model.predict(img_path, conf=0.5)
    with open(label_path) as f:
        gt_boxes = len(f.readlines())
    if len(pred[0].boxes) != gt_boxes:
        pred[0].save(filename=os.path.join(error_dir, img_file))

print(f"✅ Error cases saved to {error_dir}/")
print("✅ Training, validation, and testing completed! Model saved at /kaggle/working/forgery_Detection/weights/best.pt")

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
✅ Dataset paths verified!
YAML Content:
 augment: true
fliplr: 0.5
flipud: 0.5
mosaic: 0.5
names:
  0: forgery
nc: 1
path: /kaggle/input/forgery-dataset/Forgery Dataset
scale: 0.5
test: test/images
train: train/images
val: valid/images

✅ RT-DETR model loaded successfully!
Ultralytics 8.3.213 🚀 Python-3.11.13 torch-2.6.0+cu124 CPU (Intel Xeon CPU @ 2.20GHz)
engine/trainer: agnostic_nms=False, amp=True, augment=True, auto_augment=randaugment, batch=8, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=True, cutmix=0.0, data=/kaggle/working/c2a_rtdetr.yaml, degrees=0.0, deterministic

/usr/local/lib/python3.11/dist-packages/matplotlib/colors.py:721: RuntimeWarning: invalid value encountered in less
  xa[xa < 0] = -1
/usr/local/lib/python3.11/dist-packages/matplotlib/colors.py:721: RuntimeWarning: invalid value encountered in less
  xa[xa < 0] = -1


                   all         15         50     0.0244        0.4     0.0259    0.00876
Speed: 1.3ms preprocess, 1751.1ms inference, 0.0ms loss, 0.4ms postprocess per image
Results saved to /kaggle/working/rtdetr_forgery_detection
